In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-12-28 05:19:38--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.94MB/s    in 0.2s    

2022-12-28 05:19:38 (4.94 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_Review_Analysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_DVD_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   27288431| R33UPQQUZQEM8|B005T4ND06|     400024643|Yoga for Movement...|       Video DVD|          5|            3|          3|   N|                Y|This was a gift f...|This was a gift f...| 2015-08-31|
|         US|   13722556|R3IKTNQQPD9662|B004EPZ070|     685335564|  Something Borrowed|       Video DVD|          5|    

In [5]:
rows= df.count()
print(f"row count : {rows}")

row count : 5069140


In [6]:
print("Distinct Count: " + str(df.distinct().count()))

Distinct Count: 5069140


In [7]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df_review= df.select(["review_id","customer_id","product_id", "product_parent", "review_date"])
df_review.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| R33UPQQUZQEM8|   27288431|B005T4ND06|     400024643| 2015-08-31|
|R3IKTNQQPD9662|   13722556|B004EPZ070|     685335564| 2015-08-31|
|R3U27V5QMCP27T|   20381037|B005S9EKCW|     922008804| 2015-08-31|
|R2TOH2QKNK4IOC|   24852644|B00FC1ZCB4|     326560548| 2015-08-31|
|R2XQG5NJ59UFMY|   15556113|B002ZG98Z0|     637495038| 2015-08-31|
|R1N1KHBRR4ZTX3|    6132474|B00X8RONBO|     896602391| 2015-08-31|
|R3OM9S0TCBP38K|   48049524|B000CEXFZG|     115883890| 2015-08-31|
|R1W4S949ZRCTBW|    3282516|B00ID8H8EW|     977932459| 2015-08-31|
|R18JL1NNQAZFV2|   51771179|B000TGJ8IU|     840084782| 2015-08-31|
|R1LP6PR06OPYUX|   31816501|B00DPMPTDS|     262144920| 2015-08-31|
| RZKBT035JA0UQ|   16164990|B00X797LUS|     883589001| 2015-08-31|
|R253N5W74SM7N3|   33386989|B00C6MXB42|     734735137| 2015-08

In [8]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R33UPQQUZQEM8|          5|            3|          3|   N|                Y|
|R3IKTNQQPD9662|          5|            0|          0|   N|                Y|
|R3U27V5QMCP27T|          5|            1|          1|   N|                Y|
|R2TOH2QKNK4IOC|          5|            0|          1|   N|                Y|
|R2XQG5NJ59UFMY|          5|            0|          0|   N|                Y|
|R1N1KHBRR4ZTX3|          5|            0|          0|   N|                Y|
|R3OM9S0TCBP38K|          5|            0|          0|   N|                Y|
|R1W4S949ZRCTBW|          5|            0|          0|   N|                Y|
|R18JL1NNQAZFV2|          5|            0|          0|   N|                Y|
|R1LP6PR06OPYUX|          4|            0|          0|   N|     

In [9]:
vine_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)



In [10]:
total_votes_greater_20= vine_df.filter("total_votes>=20")
total_votes_greater_20.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R31EUR60FV3BU5|          4|           32|         39|   N|                N|
|R12Q1NO1HI9PP4|          1|            4|         86|   N|                N|
|R3MDX24QDAT0OW|          4|           35|         37|   N|                N|
|R2YVFZB4ESDT7D|          5|          139|        141|   N|                Y|
| R6TNONSNFM82I|          1|            0|         28|   N|                Y|
|R2R8EWH1JUZCSI|          5|           16|         21|   N|                N|
| RUNB838XKMB59|          1|            8|         23|   N|                N|
|R3I8GORXGEJL3R|          4|           95|        103|   N|                Y|
|R2WFUSIPJ3X0FR|          1|            9|         53|   N|                N|
| RONWXF0J85CRR|          5|           42|         50|   N|     

In [11]:
helpful_votes_greater_50_percent= total_votes_greater_20.filter("helpful_votes/total_votes >= 0.5")
helpful_votes_greater_50_percent.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R31EUR60FV3BU5|          4|           32|         39|   N|                N|
|R3MDX24QDAT0OW|          4|           35|         37|   N|                N|
|R2YVFZB4ESDT7D|          5|          139|        141|   N|                Y|
|R2R8EWH1JUZCSI|          5|           16|         21|   N|                N|
|R3I8GORXGEJL3R|          4|           95|        103|   N|                Y|
| RONWXF0J85CRR|          5|           42|         50|   N|                N|
|R2QQJIJZIUSQBH|          1|           16|         20|   N|                N|
|R3O7T74HRIXBE5|          5|           53|         55|   N|                Y|
| RSU37FK0W4DPE|          4|           26|         28|   N|                N|
|R22H100N6EUD0Y|          5|           30|         32|   N|     

In [12]:
vine_paid= helpful_votes_greater_50_percent.filter("vine == 'Y'")
vine_paid.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3PBQ665MBL8CP|          3|           66|         72|   Y|                N|
|R3EMD30K7MNOZ0|          2|           20|         25|   Y|                N|
|R2GF3ZLO2B0Y71|          5|          236|        243|   Y|                N|
|R1F8T2Z7U3N6HL|          3|           11|         20|   Y|                N|
| R5QTOT1GVR8RG|          4|           16|         20|   Y|                N|
|R1KAQEESF0NUGV|          4|           27|         29|   Y|                N|
|R1621B9AMVWI25|          2|           27|         29|   Y|                N|
|R3FVGQVNBAZL8E|          3|           37|         44|   Y|                N|
|R26ATTA765WHVX|          5|           21|         23|   Y|                N|
| RMZYPFKES1ZK7|          4|           29|         33|   Y|     

In [13]:
vine_unpaid= helpful_votes_greater_50_percent.filter("vine == 'N'")
vine_unpaid.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R31EUR60FV3BU5|          4|           32|         39|   N|                N|
|R3MDX24QDAT0OW|          4|           35|         37|   N|                N|
|R2YVFZB4ESDT7D|          5|          139|        141|   N|                Y|
|R2R8EWH1JUZCSI|          5|           16|         21|   N|                N|
|R3I8GORXGEJL3R|          4|           95|        103|   N|                Y|
| RONWXF0J85CRR|          5|           42|         50|   N|                N|
|R2QQJIJZIUSQBH|          1|           16|         20|   N|                N|
|R3O7T74HRIXBE5|          5|           53|         55|   N|                Y|
| RSU37FK0W4DPE|          4|           26|         28|   N|                N|
|R22H100N6EUD0Y|          5|           30|         32|   N|     

In [14]:
total_review_count=helpful_votes_greater_50_percent.select(["review_id"]).count()
print(f"Total number of reviews : {total_review_count}")

Total number of reviews : 151449


In [24]:
five_star_ratings=helpful_votes_greater_50_percent.filter("star_rating == 5").count()
print(f"Five star ratings count: {five_star_ratings}")

Five star ratings count: 78070


In [25]:
five_star_paid=vine_paid.filter("star_rating == 5").count()
print(f"Five star paid count: {five_star_paid}")

Five star paid count: 9


In [26]:
five_star_unpaid=vine_unpaid.filter("star_rating == 5").count()
print(f"Five star unpaid count: {five_star_unpaid}")

Five star unpaid count: 78061


In [27]:
paid_reviews_count=vine_paid.count()
print(f"Paid Reviews : {paid_reviews_count}")

Paid Reviews : 49


In [28]:
unpaid_reviews_count=vine_unpaid.count()
print(f"Unpaid Reviews : {unpaid_reviews_count}")

Unpaid Reviews : 151400


In [29]:
percentage_vine_paid = five_star_paid/paid_reviews_count
percentage_vine_paid

0.1836734693877551

In [30]:
percentage_vine_unpaid = five_star_unpaid/unpaid_reviews_count
percentage_vine_unpaid

0.5155944517833554

In [31]:
percentage_total_five_star = five_star_ratings/total_review_count
percentage_total_five_star

0.5154870616511169

In [32]:
helpful_votes_greater_50_percent.select('star_rating').describe().show()

+-------+------------------+
|summary|       star_rating|
+-------+------------------+
|  count|            151449|
|   mean|3.8106293207614446|
| stddev|1.5118010964835422|
|    min|                 1|
|    max|                 5|
+-------+------------------+



In [33]:
vine_paid.select('star_rating').describe().show()

+-------+------------------+
|summary|       star_rating|
+-------+------------------+
|  count|                49|
|   mean|3.4081632653061225|
| stddev|1.1165118178755578|
|    min|                 1|
|    max|                 5|
+-------+------------------+



In [34]:
vine_unpaid.select('star_rating').describe().show()

+-------+------------------+
|summary|       star_rating|
+-------+------------------+
|  count|            151400|
|   mean| 3.810759577278732|
| stddev|1.5118976818176069|
|    min|                 1|
|    max|                 5|
+-------+------------------+



In [61]:
df_analysis = spark.createDataFrame(
    [
        ("Y",vine_paid.count(), vine_paid.filter("star_rating == 5").count(),five_star_paid/paid_reviews_count),
        ("N",vine_unpaid.count(), vine_unpaid.filter("star_rating == 5").count(),five_star_unpaid/unpaid_reviews_count),
    
    ],
    ["Vine","Total","Five Star","Percentage"]
)

In [59]:
df_analysis.show()

+----+------+---------+------------------+
|Vine| Total|Five Star|        Percentage|
+----+------+---------+------------------+
|   Y|    49|        9|0.1836734693877551|
|   N|151400|    78061|0.5155944517833554|
+----+------+---------+------------------+

